In [1]:
!pwd

/workspaces/NER-project/notebooks


In [2]:
from pathlib import Path
from datasets import load_dataset
import pandas as pd
import torch
from transformers import AutoTokenizer, DataCollatorForTokenClassification, AutoModelForTokenClassification, pipeline
import evaluate
import numpy as np

from huggingface_hub import notebook_login
import wandb

/root/miniconda3/envs/training-pipeline/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
DATA_DIR = Path("../data/downloaded/wnut_17")
MODEL_DIR = Path("models/NER_model")

In [4]:
from datasets import load_dataset
wnut = load_dataset("wnut_17", cache_dir=DATA_DIR)

In [5]:
train_df = pd.DataFrame(wnut['train'])
train_df.tokens.apply(len).max()

41

In [6]:
label_list = wnut["train"].features[f"ner_tags"].feature.names
label_list

['O',
 'B-corporation',
 'I-corporation',
 'B-creative-work',
 'I-creative-work',
 'B-group',
 'I-group',
 'B-location',
 'I-location',
 'B-person',
 'I-person',
 'B-product',
 'I-product']

In [7]:
tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased')

In [8]:
example = wnut["train"][0]
tokenized_input = tokenizer(example["tokens"], is_split_into_words = True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])

In [9]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [10]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1287 [00:00<?, ? examples/s]

Map: 100%|██████████| 1287/1287 [00:00<00:00, 11920.08 examples/s]


In [11]:
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [12]:
import evaluate
seqeval = evaluate.load("seqeval")

In [13]:
labels = [label_list[i] for i in example[f"ner_tags"]]
labels

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-location',
 'I-location',
 'I-location',
 'O',
 'B-location',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [14]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
}

In [15]:
id2label = {
    0: "O",
    1: "B-corporation",
    2: "I-corporation",
    3: "B-creative-work",
    4: "I-creative-work",
    5: "B-group",
    6: "I-group",
    7: "B-location",
    8: "I-location",
    9: "B-person",
    10: "I-person",
    11: "B-product",
    12: "I-product",
    }
label2id = {
    "O": 0,
    "B-corporation": 1,
    "I-corporation": 2,
    "B-creative-work": 3,
    "I-creative-work": 4,
    "B-group": 5,
    "I-group": 6,
    "B-location": 7,
    "I-location": 8,
    "B-person": 9,
    "I-person": 10,
    "B-product": 11,
    "I-product": 12,
}

In [16]:
model = AutoModelForTokenClassification.from_pretrained(
    'distilbert/distilbert-base-uncased',
    num_labels = 13,
    id2label = id2label,
    label2id= label2id,
)

Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
import os
os.environ["WANDB_PROJECT"]="NER"
os.environ["WANDB_LOG_MODEL"]="end"
os.environ["WANDB_NOTEBOOK_NAME"]="POC"

In [18]:
import wandb
wandb.login()

wandb: WARNING WANDB_NOTEBOOK_NAME should be a path to a notebook file, couldn't find POC.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [21]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = "models/distilbert-ner",
    learning_rate = 2e-5,
    per_device_train_batch_size = 16,
    per_device_eval_batch_size=16,
    num_train_epochs = 2,
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    load_best_model_at_end = True,
    report_to="wandb",
    logging_steps=1
)

/root/miniconda3/envs/training-pipeline/lib/python3.10/site-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
from transformers import Trainer

trainer = Trainer(
    model = model,
    train_dataset = tokenized_wnut["train"],
    eval_dataset = tokenized_wnut["test"],
    tokenizer = tokenizer,
    data_collator = data_collator,
    compute_metrics = compute_metrics,
    args = training_args
)

In [23]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: phamlehoangvu97 (thomas24). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.502800,0.276304,0.543689,0.259500,0.351317,0.939293
2,0.078500,0.270433,0.594872,0.322521,0.418269,0.942841


/root/miniconda3/envs/training-pipeline/lib/python3.10/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=426, training_loss=0.20578305163538793, metrics={'train_runtime': 41.8068, 'train_samples_per_second': 162.366, 'train_steps_per_second': 10.19, 'total_flos': 91781128898820.0, 'train_loss': 0.20578305163538793, 'epoch': 2.0})

In [24]:
wandb.finish()

eval/accuracy,▁█
eval/f1,▁█
eval/loss,█▁
eval/precision,▁█
eval/recall,▁█
eval/runtime,█▁
eval/samples_per_second,▁█
eval/steps_per_second,▁█
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/grad_norm,█▅▂▁▁▁▂▂▃▁▁▁▁▁▂▁▂▂▂▂▂▁▁▂▁▂▁▂▂▃▂▂▂▃▁▁▂▁▂▂


In [26]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
import torch
import numpy as np

model_checkpoint = "models/distilbert-ner/checkpoint-213"
save_directory = "../onnx/"

# Load a model from transformers and export it to ONNX
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [27]:
text = "[CLS] Who was Jim Henson ? [SEP] Jim Henson was a puppeteer [SEP]"
dummy_input = tokenizer(text, return_tensors="pt")

In [29]:
torch.onnx.export(
    model, 
    tuple(dummy_input.values()),
    f="../onnx/torch-model.onnx",  
    input_names=['input_ids', 'attention_mask'], 
    output_names=['logits'], 
    dynamic_axes={'input_ids': {0: 'batch_size', 1: 'sequence'}, 
                  'attention_mask': {0: 'batch_size', 1: 'sequence'}, 
                  'logits': {0: 'batch_size', 1: 'sequence'}}, 
    do_constant_folding=True, 
    opset_version=13, 
)

In [30]:
import onnx
model_onnx = onnx.load("../onnx/torch-model.onnx")
onnx.checker.check_model(model_onnx)

In [31]:
text2 = "My name is Tom, I live in New York and my girlfriend's name is Elaine. Our parents live in Viet Nam, Nha Trang city, and their names are Que and Mai"
text2 = tokenizer(text2, return_tensors='pt')

In [32]:
import onnxruntime as ort
import numpy as np
ort_sess = ort.InferenceSession('../onnx/torch-model.onnx')
outputs = ort_sess.run(None, {'input_ids': text2['input_ids'].numpy(),
                            'attention_mask':  text2['attention_mask'].numpy()})

label = np.argmax(outputs, axis=3)
pred_labels = [label_list[i] for i in label[0][0]]
pred_labels

['O',
 'O',
 'O',
 'O',
 'B-person',
 'O',
 'O',
 'O',
 'O',
 'B-location',
 'B-location',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-person',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-location',
 'B-location',
 'O',
 'B-location',
 'O',
 'B-location',
 'B-location',
 'B-location',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-person',
 'O',
 'B-person',
 'O']

In [37]:
# import boto3
# s3_client = boto3.client('s3')
# s3_resource = boto3.resource('s3')

In [38]:
# def create_bucket(bucket_name, s3_connection):
#     session = boto3.session.Session()
#     current_region = session.region_name
#     bucket_response = s3_connection.create_bucket(
#         Bucket=bucket_name,
#         CreateBucketConfiguration={
#         'LocationConstraint': current_region})
#     print(bucket_name, current_region)
#     return bucket_name, bucket_response

In [41]:
# first_bucket_name, first_response = create_bucket(
#     bucket_name='hoangvu-ner-assets', 
#     s3_connection=s3_resource.meta.client)


# first_response

hoangvu-ner-assets ap-southeast-1


{'ResponseMetadata': {'RequestId': 'RT56EHGQEM28GS4Q',
  'HostId': 'qucOyvpj0g//6uR69RYRgI1fh1I6z5QvKuJwE66RXiKB9YXnIeDbhiEpnGSBWW8j+ijXTe0tCtY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'qucOyvpj0g//6uR69RYRgI1fh1I6z5QvKuJwE66RXiKB9YXnIeDbhiEpnGSBWW8j+ijXTe0tCtY=',
   'x-amz-request-id': 'RT56EHGQEM28GS4Q',
   'date': 'Mon, 22 Jul 2024 14:04:42 GMT',
   'location': 'http://hoangvu-ner-assets.s3.amazonaws.com/',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'Location': 'http://hoangvu-ner-assets.s3.amazonaws.com/'}